<a href="https://colab.research.google.com/github/chagas98/BioinfoWikidatascripts/blob/main/wiki_bioinfo_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Bibliotecas**



In [ ]:
import numpy as np    
import pandas as pd
from google.colab import data_table

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()




### **Dados de Entrada**

Link dos arquivos .csv disponíveis nos Dados Abertos da CAPES. Classificados em: Programas (prog), Discentes (stud) e Docentes (prof) (dados 2017-2021)



In [ ]:
 csv_links = {"prog": "https://dadosabertos.capes.gov.br/dataset/903b4215-ea91-4927-8975-d1484891374f/resource/9835dd45-d4e7-4b1f-b550-eb9b049bacac/download/br-capes-colsucup-prog-2017-2021-11-10.csv",
"stud": "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/e9f13006-d0d3-4869-9790-1a8f5d31cf0c/download/br-capes-colsucup-discentes-2017-2021-11-10.csv",
"prof": "https://dadosabertos.capes.gov.br/dataset/57f86b23-e751-4834-8537-e9d33bd608b6/resource/89662c04-3451-45cd-b2e8-ae430bc6e00f/download/br-capes-colsucup-docente-2017-2021-11-10.csv"}
csv_links

In [ ]:
data_table.enable_dataframe_formatter()


#### **Google Sheets**

Rodar apenas uma vez, cria uma tabela para armazenar informações extraídas.

In [ ]:
gc = gspread.authorize(creds)

spreadsheet_name = "bioinfo_wiki_brasil"
sh = gc.create(spreadsheet_name)

sh = gc.open(spreadsheet_name)
print("https://docs.google.com/spreadsheets/d/%s" % sh.id)

###**Programas Bioinfo**

Extração de informações a partir dos programas de bioinfo cadastrados pela CAPES.

**Seleção de Informações**


*   Universidade
*   Código Programa
*   Cidade
*   Estado
*   Região

In [ ]:
dataprog = pd.read_csv(csv_links.get("prog"), sep = ";", encoding = "ISO-8859-1")

bioinfo_prog = dataprog[dataprog["NM_PROGRAMA_IES"].str.contains("BIOINFO")]

In [ ]:
select_info = ["NM_PROGRAMA_IES", "NM_PROGRAMA_IDIOMA","NM_ENTIDADE_ENSINO", "CD_PROGRAMA_IES", "NM_MUNICIPIO_PROGRAMA_IES", "SG_UF_PROGRAMA", "NM_REGIAO"]
bioinfo_progsel = bioinfo_prog.loc[:, select_info]

bioinfo_progsel["Name"] =  bioinfo_progsel["NM_PROGRAMA_IDIOMA"] + " (" + bioinfo_progsel["NM_ENTIDADE_ENSINO"] + ")"

bioinfo_progsel


,NM_PROGRAMA_IES,NM_PROGRAMA_IDIOMA,NM_ENTIDADE_ENSINO,CD_PROGRAMA_IES,NM_MUNICIPIO_PROGRAMA_IES,SG_UF_PROGRAMA,NM_REGIAO,Name
522,BIOINFORMÁTICA,BIOINFORMATICS,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,23001011170P8,NATAL,RN,NORDESTE,BIOINFORMATICS (UNIVERSIDADE FEDERAL DO RIO GR...
1760,BIOINFORMÁTICA,POST GRADUATE PROGRAM IN BIOINFORMATICS,UNIVERSIDADE FEDERAL DE MINAS GERAIS,32001010068P4,BELO HORIZONTE,MG,SUDESTE,POST GRADUATE PROGRAM IN BIOINFORMATICS (UNIVE...
2214,BIOINFORMÁTICA,INTER-DEPARTMENTAL GRADUATE PROGRAM IN BIOINFO...,UNIVERSIDADE DE SÃO PAULO,33002010188P9,SÃO PAULO,SP,SUDESTE,INTER-DEPARTMENTAL GRADUATE PROGRAM IN BIOINFO...
3130,BIOINFORMÁTICA,BIOINFORMATICS,UNIVERSIDADE FEDERAL DO PARANÁ,40001016066P4,CURITIBA,PR,SUL,BIOINFORMATICS (UNIVERSIDADE FEDERAL DO PARANÁ)
3287,BIOINFORMÁTICA,BIOINFORMATICS GRADUATE PROGRAM,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,40006018037P6,CORNÉLIO PROCÓPIO,PR,SUL,BIOINFORMATICS GRADUATE PROGRAM (UNIVERSIDADE ...


In [ ]:
df_prog = pd.DataFrame({"names": bioinfo_progsel["Name"].to_list()})

worksheet_prog = sh.sheet1
#worksheet_prog = sh.add_worksheet("Programas", rows = "100", cols = "9")

worksheet_prog.update_cell(1, 1, df_prog.columns[0])
worksheet_prog.update_cell(1, 2, "wikidata_id")
worksheet_prog.update_cell(1, 3, "blank")
worksheet_prog.update_cell(1, 4, "wikidata_description")

for i, row in df_prog.iterrows():
  worksheet_prog.update_cell(i+2, 1, row[0])
  worksheet_prog.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_prog.update_cell(1, 3, "")
  worksheet_prog.update_cell(i+2, 4, "=WIKIDATADESCRIPTIONS(B"+ str(i+2)+")")

###**Discentes Bioinfo**

Seleção dos discentes que receberam título em um programa de PG em Bioinfo.

**Seleção de Informações**

*   Universidade
*   Nacionalidade
*   Código Programa
*   Orientador(a)
*   Grau
*   Tese

In [ ]:
datastud = pd.read_csv(csv_links.get("stud"), sep = ";", encoding = "ISO-8859-1")

bioinfo_stud = datastud[datastud["NM_PROGRAMA_IES"].str.contains("BIOINFO") & datastud["NM_SITUACAO_DISCENTE"].str.contains("TITULADO")]


In [ ]:
select_info = ["NM_DISCENTE","NM_ENTIDADE_ENSINO","DS_TIPO_NACIONALIDADE_DISCENTE","CD_PROGRAMA_IES","NM_ORIENTADOR_PRINCIPAL", "DS_GRAU_ACADEMICO_DISCENTE", "NM_TESE_DISSERTACAO"]
bioinfo_studsel = bioinfo_stud.loc[:, select_info]


bioinfo_studsel


,NM_DISCENTE,NM_ENTIDADE_ENSINO,DS_TIPO_NACIONALIDADE_DISCENTE,CD_PROGRAMA_IES,NM_ORIENTADOR_PRINCIPAL,DS_GRAU_ACADEMICO_DISCENTE,NM_TESE_DISSERTACAO
34574,IARA DANTAS DE SOUZA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,RODRIGO JULIANI SIQUEIRA DALMOLIN,MESTRADO,MAPA METABÓLICO DA INTOXICAÇÃO POR CHUMBO
151151,RAFAEL EDUARDO OLIVEIRA ROCHA,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,RAFAELA SALGADO FERREIRA,MESTRADO,ESTUDOS DE MODELAGEM MOLECULAR DOS MECANISMOS ...
151184,FRANCIELLY MORAIS RODRIGUES DA COSTA,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,MARCOS AUGUSTO DOS SANTOS,DOUTORADO,MICROARRAY-BASED BREAST CANCER CLASSIFICATION ...
151314,FENICIA BRITO SANTOS,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,JOSE MIGUEL ORTEGA,MESTRADO,ORIGEM DOS GENES DAS VIAS DE SECREÇÃO DO SISTE...
151326,NAYARA EVELIN DE TOLEDO,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,GLORIA REGINA FRANCO,MESTRADO,OCORRÊNCIA DE SPLICING ALTERNATIVO DIFERENCIAL...
...,...,...,...,...,...,...,...
292705,ERINALDO SANCHES NASCIMENTO,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,ALESSANDRO BOTELHO BOVO,MESTRADO,DESENVOLVIMENTO DE UMA FERRAMENTA WEB PARA DES...
292706,CYNARA LEAO GARCIA,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,ANDRE YOSHIAKI KASHIWABARA,MESTRADO,ANALISANDO REGIÕES CODIFICADORAS DE PROTEÍNAS ...
292719,JADER MAIKOL CALDONAZZO GARBELINI,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,DANILO SIPOLI SANCHES,MESTRADO,ABORDAGEM BASEADA EM ALGORÍTIMOS MEMÉTICOS PAR...
292720,TATIANNE DA COSTA NEGRI,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,ALEXANDRE ROSSI PASCHOAL,MESTRADO,RNAPLONC: UM CLASSIFICADOR PARA IDENTIFICAÇÃO ...


In [ ]:
df_stud = pd.DataFrame({"names": bioinfo_studsel["NM_DISCENTE"].to_list()})


worksheet_stud = sh.add_worksheet("Discentes", rows = "300", cols = "10")

worksheet_stud.update_cell(1, 1, df_stud.columns[0])
worksheet_stud.update_cell(1, 2, "wikidata_id")
worksheet_stud.update_cell(1, 3, "blank")
worksheet_stud.update_cell(1, 4, "wikidata_description")

for i, row in df_stud.iterrows():
  worksheet_stud.update_cell(i+2, 1, row[0])
  worksheet_stud.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_stud.update_cell(1, 3, "")
  worksheet_stud.update_cell(i+2, 4, "=WIKIDATADESCRIPTIONS(B"+ str(i+2)+")")

###**Docentes Bioinfo**

Seleção de docentes que estão ligados a um programa de PG em Bioinfo. 


**Seleção de Informações**

*   Universidade
*   Nacionalidade
*   Código Programa
*   Última Titulação
*   Origem

In [ ]:
dataprof = pd.read_csv(csv_links.get("prof"), sep = ";", encoding = "ISO-8859-1")

bioinfo_prof = dataprof[dataprof["NM_PROGRAMA_IES"].str.contains("BIOINFO")]

In [ ]:
select_info = ["NM_DOCENTE","NM_ENTIDADE_ENSINO","DS_TIPO_NACIONALIDADE_DOCENTE","CD_PROGRAMA_IES","NM_AREA_BASICA_TITULACAO", "NM_IES_TITULACAO"]
bioinfo_profsel = bioinfo_prof.loc[:, select_info]


bioinfo_profsel

,NM_DOCENTE,NM_ENTIDADE_ENSINO,DS_TIPO_NACIONALIDADE_DOCENTE,CD_PROGRAMA_IES,NM_AREA_BASICA_TITULACAO,NM_IES_TITULACAO
10495,EUZEBIO GUIMARAES BARBOSA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,QUÍMICA,UNIVERSIDADE ESTADUAL DE CAMPINAS
10501,GUSTAVO ANTONIO DE SOUZA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,MORFOLOGIA,UNIVERSIDADE DE SÃO PAULO ( RIBEIRÃO PRETO )
10550,RODRIGO JULIANI SIQUEIRA DALMOLIN,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,BIOQUÍMICA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
10615,DANIEL SABINO AMORIM DE ARAUJO,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,ENGENHARIA ELÉTRICA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
10619,JORGE ESTEFANO SANTANA DE SOUZA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,GENÉTICA,UNIVERSIDADE DE SÃO PAULO
...,...,...,...,...,...,...
79857,ALEXANDRE ROSSI PASCHOAL,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,METODOLOGIA E TÉCNICAS DA COMPUTAÇÃO,UNIVERSITÄT LEIPZIG
79873,LAURIVAL ANTONIO VILAS BOAS,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,MELHORAMENTO VEGETAL,UNIVERSIDADE ESTADUAL PAULISTA JÚLIO DE MESQUI...
79888,PEDRO HENRIQUE BUGATTI,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,CIÊNCIA DA COMPUTAÇÃO,UNIVERSIDADE DE SÃO PAULO
79982,HEITOR SILVERIO LOPES,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,BRASILEIRO,40006018037P6,ENGENHARIA ELÉTRICA,UNIVERSIDADE FEDERAL DE SANTA CATARINA


In [ ]:
df_prof = pd.DataFrame({"names": bioinfo_profsel["NM_DOCENTE"].to_list()})


worksheet_prof = sh.add_worksheet("Docentes", rows = "300", cols = "10")

worksheet_prof.update_cell(1, 1, df_prof.columns[0])
worksheet_prof.update_cell(1, 2, "wikidata_id")
worksheet_prof.update_cell(1, 3, "blank")
worksheet_prof.update_cell(1, 4, "wikidata_description")

for i, row in df_prof.iterrows():
  worksheet_prof.update_cell(i+2, 1, row[0])
  worksheet_prof.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_prof.update_cell(1, 3, "")
  worksheet_prof.update_cell(i+2, 4, "=WIKIDATADESCRIPTIONS(B"+ str(i+2)+")")